# Lecture 12: Data Preparation I

In [ ]:
# to save space, do a symbolic link to previous lecture's data
!ln -sf ../lec11/data .

## [review] Structural Transformation: From Relations to Matrices and Back
- Matrix $\rightarrow$ Relational works.
- Relational $\rightarrow$ Matrix sometimes works!
- But how?

To start, let's take our matrix in `mm.txt`, and load it into Pandas.

In [ ]:
import pandas as pd

In [ ]:
mm = pd.read_csv('data/mm.txt', header=0)
mm

## [Review] What does an unpivot look like (Matrix -> Relational)?

In [ ]:
mm_melted = mm.melt(id_vars=['Year'])
mm_melted

Thanks to the `id_var` parameter, the `Year` column is named and repeated for all other (variable=column name, value=value) elements in the row.

In [ ]:
mm_melted[mm_melted['Year'] == 2002]

## PIVOT(UNPIVOT) = ??

In [ ]:
# how do we get back to something that resembles the original matrix?

# Data Unboxing, Part I

In [ ]:
!du -h data/*

What category of data is the file below? Any observations about the data?

In [ ]:
!head -c 1024 data/jc1.txt

**Exercise**: What do you see this time? Category? Interesting features of the data?

In [ ]:
!head -c 1024 data/jq2.txt

## Data Unboxing, Part II

Let's explore the first file here:

In [ ]:
!head -c 1024 data/jc1.txt

In [ ]:
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

jc1 = pd.read_csv('data/jc1.txt', header=0, sep=',')
jc1.head(3)

What data types are each column?

In [ ]:
jc1.dtypes

Currently, we're working with objects, because of how pandas read in the dtypes. Let's try to modify some scores here for contestants to make them numbers

In [ ]:
jc1.columns
for col in jc1.columns:
    if '_score' in col:
        jc1[col] = jc1[col].map(lambda x: float(x.strip('$')) if type(x) == str else np.nan)

What are our dtypes now?

In [ ]:
jc1.dtypes

Can we visualize our contestant winnings?

In [ ]:
jc1.hist(column='contestant1_score')
jc1.hist(column='contestant2_score')
jc1.hist(column='contestant3_score')

# CNRFC-NOAA Rainfall Dataset

* There are four data files tied to the scrape of rainfall data from the NOAA (National Oceanic and Atmospheric Administration) [link](https://www.cnrfc.noaa.gov/monthly_precip_2020.php).
* This was compiled by visiting each annual website of monthly precipitation by regions in the California Nevada River Forecast Center (CNRFC) area.
* For more details on how to make this dataset yourself, check out the `rainfallscrape` folder. You may need to install additional Python packages via `pip -r rainfallscrape/requirements.txt`.

Let's unbox this data too:

In [ ]:
!ls -lh data/m*

All of these files *look* rectangular. **What data model does each follow, structurally?**

Analyze the first few lines of each file in order below. Remember to adjust `-n` number of lines as needed.
1. `mm.txt`
2. `mmp.txt`
3. `mmr.txt`
4. `mpf.txt` - may need to look at first 20 lines here

In [ ]:
!head -n 20 data/mpf.txt

## Fulfilling Structural Transformation Requests

Let's start from the long (tidy, relational) file:

In [ ]:
pd.set_option('display.min_rows', 15) # changes truncated view in pandas

In [ ]:
mmr = pd.read_csv('data/mmr.txt')
mmr

A data analyst asks for just the precipitation grouped by year and month, with no location data. **How do we do this?**
* What are they asking for? This isn't intuitive, because we have location data as well. What do we do?
* How do we pivot the data into year x month form?

In [ ]:
# fill in here

<br/><br/><br/><br/>

### solution

In [ ]:
mmr_grouped = mmr_grouped.reset_index()
mmr_grouped

In [ ]:
mmr_pivoted = mmr_grouped.pivot(index='Year', columns='Month')
mmr_pivoted

---

# Scalar Functions and Query Plans

In [ ]:
## we'll use the Lahman baseball database in our examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://localhost:5432/baseball
%config SqlMagic.displaylimit = 30

In [ ]:
%%sql
WITH yearnum AS
  (SELECT yearid, (yearid % 100) as year
     FROM batting
  )
SELECT yearid, CONCAT('''', LPAD(year::text, 2, '0')) as year
  FROM yearnum
 LIMIT 5;

Let's analyze the below query (we've flattened it for convenience):

In [ ]:
%%sql
EXPLAIN (VERBOSE true)
SELECT yearid,
       CONCAT('''', LPAD((yearid % 100)::text, 2, '0')) AS year
FROM batting;


What if scalar functions mention multiple tables?

The below query computes an arbitrary statistic for pitchers:
* 1 point for every strikeout they throw as pitcher
* –1 for every point they themselves struck out as batter

If the notebook-like output is hard to read, try out the query in `psql`. Note that notebooks don't preserve whitespace when displaying dataframes.

In [ ]:
%%sql
EXPLAIN (VERBOSE true)
SELECT p.playerid, p.so - b.so
  FROM pitching p
  INNER JOIN batting b
  ON p.playerid=b.playerid;

## [Extra] Recreating the CNRFC-NOAA data

* There are four data files tied to the scrape of rainfall data from the NOAA (National Oceanic and Atmospheric Administration) [link](https://www.cnrfc.noaa.gov/monthly_precip_2020.php).
* This was compiled by visiting each annual website of monthly precipitation by regions in the California Nevada River Forecast Center (CNRFC) area.
* For more details on how to make this dataset yourself, check out the `rainfallscrape` folder. You may need to install additional Python packages via `pip -r rainfallscrape/requirements.txt`. We'll assume you've already run `simple_scrape.py`.

In [ ]:
!head -c 4096 data/mpf.txt

Messy! You can play in bash or pandas if you like. Let's clean this up a bit. Firstly, we note that the structure of the file is a bit off. It's hard to read multiple dataframes from a single CSV like this...so let's try to write multiple files and concatenate a df.

- Going to iterate through the lines in this files...
- Identify the files with names of locations
- print the filenames transformed to see what it's like.

In [ ]:
curr_file = None
with open('data/mpf.txt') as f:
    for line in f.readlines():
        splits = line.split(', ')
        if len(splits) == 2:
            print(splits[1])

Okay, first, let's create some files; one for each segment. And let's rename them into something more conventional. 

- Going to iterate through the lines in this files...
- Identify the files with names of locations
- print the filenames transformed to see what it's like.

In [ ]:
curr_file = None
with open('lec12_data/mpf.txt') as f:
    for num, line in enumerate(f.readlines()):
        splits = line.split(', ')
        if len(splits) == 2:
            print(splits[1].strip(" '\n ").replace('...', '-').replace(' ', '_'))

That's probably more of what we want! Now, let's actually write our new files...

In [ ]:
curr_file = None
with open('data/mpf.txt') as master_file:
    for line in master_file.readlines():
        splits = line.split(', ')
        if len(splits) == 2:
            if curr_file is not None:
                curr_file.close()
            filename = splits[0] + '_' + splits[1].strip(" '\n ")
            filename = filename.replace('...', '-')
            filename = filename.replace(' ', '_')
            curr_file = open('data/subfiles/' + filename + '.txt', 'w')
        curr_file.write(line)
curr_file.close()

Now, let's read these files into a dataframe...and we'll need to do a lot of preprocessing!

In [ ]:
import os
mpf = pd.DataFrame()
parent_dir = os.getcwd() + '/data/subfiles/'
for filename in os.listdir(parent_dir):
    # read from the first header. not the second!
    region_df = pd.read_csv(parent_dir + '/' + filename, header=1)
    # BE CAREFUL! Let's rename our schema...because right now, this is problematic.
    # The first column is just labeled as the year itself (e.g. 2007). Not 'Year'.
    region_df = region_df.rename(columns={filename.split('_')[0]: 'Year'})
    mpf = mpf.append(region_df)
mpf = mpf.sort_values('Year')
mpf

Now, let's figure out what our data looks like in Pandas.

In [ ]:
mpf

In [ ]:
mpf.dtypes

Let's rename our columns!

In [ ]:
col_set = {}
for column in mpf.columns:
    if column != 'Year':
        col_set.update({column: column.strip(" '")})
mpf = mpf.rename(columns=col_set)
mpf

In [ ]:
# Let's remove the Ms and replace them with NaNs, and strip the quotes
import math
def _process_data(x):
    if type(x) == float:
        return x
    if type(x) != str and math.isnan(x):
        return np.nan
    x = x.strip("' ")
    if x == 'M' or x == 'NA':
        return np.nan
    else:
        return float(x)

mpf = mpf[mpf['Year'] == 2002]
for col in mpf.columns:
    if col in ["ID", "Location"]:
        print(col)
        mpf[col] = mpf[col].map(lambda x: str(x).strip("'"))
    elif col != 'Year':
        mpf[col] = mpf[col].map(lambda x: _process_data(x))
mpf